# Chapter 11: Goal Setting and Monitoring

Key Takeaways:
- **Goal-Driven Iteration**: Define explicit goals and iterate until they are met.
- **Feedback Loops**: Use an LLM to critique generated outputs and provide actionable feedback.
- **LLM-as-Evaluator**: The same LLM can act as both generator and evaluator to determine goal completion.
- **Iterative Refinement**: Each iteration builds upon the previous output, guided by feedback.

### Heuristic: *Iterate until goals are met.*

## Setup and Initialization

In [ ]:
import os
import sys
import random
import re
from pathlib import Path

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# Add scripts directory to path
PROJECT_ROOT = os.path.dirname(os.getcwd())
SCRIPTS_DIR = os.path.join(PROJECT_ROOT, "scripts")
sys.path.insert(0, SCRIPTS_DIR)

from goal_agent_utils import clean_code_block, add_comment_header, to_snake_case

# Load environment variables
load_dotenv()

# Verify API key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise EnvironmentError("❌ Please set the OPENAI_API_KEY environment variable.")

print(f"✅ Configuration Loaded:")
print(f"   Project Root: {PROJECT_ROOT}")
print(f"   Scripts Directory: {SCRIPTS_DIR}")

In [ ]:
# Initialize OpenAI LLM
print("📡 Initializing OpenAI LLM (gpt-4o)...")

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.3,
    openai_api_key=OPENAI_API_KEY,
)

print("✅ LLM initialized successfully")

## 1. Utility Functions

These functions handle prompt generation, feedback evaluation, and goal verification.

In [ ]:
def generate_prompt(
    use_case: str, goals: list[str], previous_code: str = "", feedback: str = ""
) -> str:
    """Construct a prompt for code generation based on use case and goals."""
    print("📝 Constructing prompt for code generation...")
    
    base_prompt = f"""
You are an AI coding agent. Your job is to write Python code based on the following use case:

Use Case: {use_case}

Your goals are:
{chr(10).join(f"- {g.strip()}" for g in goals)}
"""
    
    if previous_code:
        print("🔄 Adding previous code to the prompt for refinement.")
        base_prompt += f"\nPreviously generated code:\n{previous_code}"
    
    if feedback:
        print("📋 Including feedback for revision.")
        base_prompt += f"\nFeedback on previous version:\n{feedback}\n"
    
    base_prompt += "\nPlease return only the revised Python code. Do not include comments or explanations outside the code."
    
    return base_prompt

print("✅ generate_prompt() defined")

In [ ]:
def get_code_feedback(code: str, goals: list[str]) -> str:
    """Use the LLM to evaluate code against defined goals."""
    print("🔍 Evaluating code against the goals...")
    
    feedback_prompt = f"""
You are a Python code reviewer. A code snippet is shown below.

Based on the following goals:
{chr(10).join(f"- {g.strip()}" for g in goals)}

Please critique this code and identify if the goals are met.
Mention if improvements are needed for clarity, simplicity, correctness, edge case handling, or test coverage.

Code:
{code}
"""
    return llm.invoke(feedback_prompt)


def goals_met(feedback_text: str, goals: list[str]) -> bool:
    """
    Uses the LLM to evaluate whether the goals have been met based on the feedback text.
    Returns True or False (parsed from LLM output).
    """
    review_prompt = f"""
You are an AI reviewer.

Here are the goals:
{chr(10).join(f"- {g.strip()}" for g in goals)}

Here is the feedback on the code:
\"\"\"
{feedback_text}
\"\"\"

Based on the feedback above, have the goals been met?
Respond with only one word: True or False.
"""
    response = llm.invoke(review_prompt).content.strip().lower()
    return response == "true"

print("✅ get_code_feedback() and goals_met() defined")

In [ ]:
def save_code_to_file(code: str, use_case: str) -> str:
    """Save the generated code to a file with a descriptive name."""
    print("💾 Saving final code to file...")
    
    summary_prompt = (
        f"Summarize the following use case into a single lowercase word or phrase, "
        f"no more than 10 characters, suitable for a Python filename:\n\n{use_case}"
    )
    raw_summary = llm.invoke(summary_prompt).content.strip()
    short_name = re.sub(r"[^a-zA-Z0-9_]", "", raw_summary.replace(" ", "_").lower())[:10]
    
    random_suffix = str(random.randint(1000, 9999))
    filename = f"{short_name}_{random_suffix}.py"
    
    # Save to scripts directory
    filepath = Path(SCRIPTS_DIR) / "generated" / filename
    filepath.parent.mkdir(parents=True, exist_ok=True)
    
    with open(filepath, "w") as f:
        f.write(code)
    
    print(f"✅ Code saved to: {filepath}")
    return str(filepath)

print("✅ save_code_to_file() defined")

## 2. Main Agent Function

The core iterative loop that generates code, gets feedback, and refines until goals are met.

In [ ]:
def run_code_agent(use_case: str, goals_input: str, max_iterations: int = 5) -> str:
    """
    Run the goal-driven code generation agent.
    
    Args:
        use_case: Description of what the code should accomplish
        goals_input: Comma-separated list of goals
        max_iterations: Maximum refinement iterations
    
    Returns:
        Path to the saved code file
    """
    goals = [g.strip() for g in goals_input.split(",")]
    
    print(f"\n🎯 Use Case: {use_case}")
    print("🎯 Goals:")
    for g in goals:
        print(f"   - {g}")
    
    previous_code = ""
    feedback = ""
    
    for i in range(max_iterations):
        print(f"\n{'='*50}")
        print(f"🔁 Iteration {i + 1} of {max_iterations}")
        print(f"{'='*50}")
        
        # Generate code
        prompt = generate_prompt(
            use_case, goals, previous_code,
            feedback if isinstance(feedback, str) else feedback.content
        )
        
        print("🚧 Generating code...")
        code_response = llm.invoke(prompt)
        raw_code = code_response.content.strip()
        code = clean_code_block(raw_code)
        
        print("\n🧾 Generated Code:")
        print("-" * 50)
        print(code)
        print("-" * 50)
        
        # Get feedback
        print("\n📤 Submitting code for feedback review...")
        feedback = get_code_feedback(code, goals)
        feedback_text = feedback.content.strip()
        
        print("\n📥 Feedback Received:")
        print("-" * 50)
        print(feedback_text)
        print("-" * 50)
        
        # Check if goals are met
        if goals_met(feedback_text, goals):
            print("\n✅ LLM confirms goals are met. Stopping iteration.")
            break
        
        print("\n⚠️ Goals not fully met. Preparing for next iteration...")
        previous_code = code
    
    # Finalize and save
    final_code = add_comment_header(code, use_case)
    return save_code_to_file(final_code, use_case)

print("✅ run_code_agent() defined")

## 3. Example Usage

Let's generate code for finding the binary gap of a positive integer.

In [ ]:
print("\n🧠 Welcome to the AI Code Generation Agent")
print("="*50)

# Define the use case and goals
use_case = "Write code to find BinaryGap of a given positive integer"

goals = (
    "Code simple to understand, "
    "Functionally correct, "
    "Handles comprehensive edge cases, "
    "Takes positive integer input only, "
    "Prints the results with few examples"
)

# Run the agent
result_path = run_code_agent(use_case, goals)

print(f"\n🎉 Final code saved to: {result_path}")

### View Generated Code

In [ ]:
# Display the generated file
with open(result_path, 'r') as f:
    print(f.read())

## Conclusion

The Goal Setting and Monitoring pattern demonstrates how to use explicit goals and iterative feedback loops to guide an AI agent toward producing high-quality outputs. By leveraging the LLM as both generator and evaluator, the agent can self-correct and refine its work until the defined objectives are achieved.